<a href="https://colab.research.google.com/github/Sarthak-Jadhav-Dev/ParrallalismInAPRIORI/blob/main/ParrallalismAPRIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apriori with GPU-accelerated support counting (CUDA)
This notebook compiles and runs a CUDA-accelerated Apriori (counting step on GPU).
Steps:
1. Ensure GPU runtime is enabled (Runtime → Change runtime type → GPU).
2. Check `nvcc`. If missing, try the automatic install.
3. Compile and run the CUDA C++ program.

#Points to Note:
1.Only parallelized the support counting step (the heavy inner loop) on the GPU. Candidate generation, I/O, and rule generation stay on the host (C++).



#The Code Below is to Make a Dataset File in CSV Format of Lakh Records

In [ ]:
#The File Below is Just for making the dataset on the Google Colabs Runtime Storage
import random, csv

# Base transactions (realistic shopping baskets)
base = [
    ["bread","milk","eggs"],
    ["milk","laptop","beer","bread"],
    ["bread","butter","cheese"],
    ["mobile","headphones","powerbank"],
    ["laptop","mouse","keyboard"],
    ["shampoo","soap","toothpaste"],
    ["bread","milk","laptop"],
    ["eggs","flour","sugar","butter"],
    ["bread","butter","jam"],
    ["mobile","charger","cover"],
    ["laptop","mouse","pad","usb"],
    ["milk","cereal","bananas"],
    ["bread","milk","eggs","cheese"],
    ["chips","coke","chocolate"],
    ["rice","dal","oil","salt"],
    ["bread","eggs","flour","milk"],
    ["sofa","table","chair"],
    ["tshirt","jeans","shoes"],
    ["bread","butter","milk"],
    ["laptop","wipes","baby_powder"]
]

# Parameters
num_transactions = 100000   # make it big enough
filename = "realistic_dataset.csv"

# Write dataset
with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    for _ in range(num_transactions):
        transaction = random.choice(base)[:]     # pick a base basket
        # randomly drop/add one item to make variation
        if random.random() < 0.3:
            transaction = transaction[:-1]
        if random.random() < 0.3:
            transaction.append(random.choice(
                ["tea","coffee","cookies","fruits","yogurt","tv","fan","shoes"]
            ))
        writer.writerow(transaction)

print(f"Created {filename} with {num_transactions} realistic transactions")



Created realistic_dataset.csv with 100000 realistic transactions


#Check GPU & nvcc (runtime must be GPU)

**nvcc** : nvcc stands for NVIDIA CUDA Compiler.

1.It is the compiler that comes with the CUDA Toolkit.

2.Purpose: It compiles programs that use CUDA (Compute Unified Device Architecture) — NVIDIA’s parallel programming model for GPUs.

**nvcc handles both:**

1.CPU code (C/C++) → sends it to a normal host compiler (like g++ or clang).

2.GPU code (CUDA kernels) → compiles into GPU machine code (PTX/SASS).

**Then it links them together into one executable.**

In [ ]:
# Check GPU & nvcc presence
!nvidia-smi
!nvcc --version || echo "nvcc not found on this runtime"


Fri Sep 19 04:09:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

##The Below Code Creates a `.cu` file in the Google Colab

`.cu` is **File ending with .cu are just C++ source files that contain CUDA code.**

In [ ]:
%%bash
cat > apriori_cuda.cu <<'EOF'
/* Apriori GPU-accelerated (support counting on CUDA)
   - built from user's APRIORI.cpp as basis
   - kernel checks candidate presence using bitmask words (64-bit)
   - compile with nvcc
*/

#include <bits/stdc++.h>
#include <cuda_runtime.h>

using namespace std;
using Item = string;
using Itemset = set<Item>;
using Transactions = vector<vector<Item>>;
using FrequentItemsets = map<Itemset, double>;
using Rules = vector<tuple<Itemset, Itemset, double, double>>;

#define CUDA_CHECK(call)                                                     \
do {                                                                         \
    cudaError_t err = call;                                                  \
    if (err != cudaSuccess) {                                                \
        fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__,     \
                cudaGetErrorString(err));                                    \
        exit(EXIT_FAILURE);                                                  \
    }                                                                        \
} while(0)

vector<Item> split_csv_line(const string &line) {
    vector<Item> items;
    stringstream ss(line);
    string token;
    while (getline(ss, token, ',')) {
        auto start = token.find_first_not_of(" \n\r\t");
        auto end = token.find_last_not_of(" \n\r\t");
        if (start == string::npos) continue;
        string t = token.substr(start, end - start + 1);
        if (!t.empty()) items.push_back(t);
    }
    return items;
}

Transactions read_transactions(const string &filename) {
    ifstream file(filename);
    Transactions transactions;
    if (!file) {
        cerr << "Error: Unable to open file " << filename << endl;
        return transactions;
    }
    string line;
    while (getline(file, line)) {
        auto tr = split_csv_line(line);
        if (!tr.empty()) transactions.push_back(tr);
    }
    file.close();
    return transactions;
}

vector<Itemset> create_candidate_itemsets(const Transactions &transactions, int k) {
    set<Itemset> candidates;
    if (k == 1) {
        for (const auto &transaction : transactions) {
            for (const auto &item : transaction) {
                candidates.insert(Itemset{item});
            }
        }
    } else {
        set<Item> all_items;
        for (const auto &transaction : transactions) {
            all_items.insert(transaction.begin(), transaction.end());
        }
        vector<Item> items_vector(all_items.begin(), all_items.end());
        vector<bool> bitmask(k, true);
        bitmask.resize(items_vector.size(), false);
        do {
            Itemset candidate;
            for (int i = 0; i < (int)items_vector.size(); ++i) {
                if (bitmask[i]) candidate.insert(items_vector[i]);
            }
            if ((int)candidate.size() == k)
                candidates.insert(candidate);
        } while (prev_permutation(bitmask.begin(), bitmask.end()));
    }
    return vector<Itemset>(candidates.begin(), candidates.end());
}

/* CUDA kernel: each thread checks one (candidate, transaction) pair.
   trans_masks: num_transactions * M 64-bit words (flattened)
   cand_masks:  num_candidates  * M 64-bit words (flattened)
   counts:      num_candidates ints (atomicAdd)
*/
extern "C" __global__
void count_kernel(const unsigned long long *trans_masks,
                  const unsigned long long *cand_masks,
                  int M,
                  int num_transactions,
                  int num_candidates,
                  int *counts) {
    long long idx = (long long)blockIdx.x * blockDim.x + threadIdx.x;
    long long total = (long long)num_candidates * (long long)num_transactions;
    if (idx >= total) return;
    int cand_idx = (int)(idx / num_transactions);
    int trans_idx = (int)(idx % num_transactions);
    const unsigned long long *t = trans_masks + (long long)trans_idx * M;
    const unsigned long long *c = cand_masks + (long long)cand_idx * M;
    bool ok = true;
    for (int i = 0; i < M; ++i) {
        if ((t[i] & c[i]) != c[i]) { ok = false; break; }
    }
    if (ok) atomicAdd(&counts[cand_idx], 1);
}

FrequentItemsets filter_itemsets_gpu(const Transactions &transactions,
                                     const vector<Itemset> &candidates,
                                     double min_support,
                                     const unordered_map<Item,int> &item2id,
                                     int M,
                                     const vector<unsigned long long> &trans_masks) {
    int num_transactions = (int)transactions.size();
    int num_candidates = (int)candidates.size();
    vector<int> counts(num_candidates, 0);
    if (num_candidates == 0) return {};

    // Build candidate masks on host
    vector<unsigned long long> cand_masks((size_t)num_candidates * M, 0ULL);
    for (int cidx = 0; cidx < num_candidates; ++cidx) {
        for (const auto &it : candidates[cidx]) {
            auto itid = item2id.find(it);
            if (itid == item2id.end()) continue;
            int id = itid->second;
            int word = id / 64;
            int bit = id % 64;
            cand_masks[(size_t)cidx * M + word] |= (1ULL << bit);
        }
    }

    // Device allocation for transactions (only once)
    unsigned long long *d_trans = nullptr;
    CUDA_CHECK(cudaMalloc(&d_trans, (size_t)num_transactions * M * sizeof(unsigned long long)));
    CUDA_CHECK(cudaMemcpy(d_trans, trans_masks.data(), (size_t)num_transactions * M * sizeof(unsigned long long), cudaMemcpyHostToDevice));

    // Batch candidates to limit memory and threads
    const long long max_pairs = 10000000LL; // tune this value if needed
    int batch_candidates = max(1, (int)min((long long)num_candidates, max_pairs / max(1, num_transactions)));
    size_t offset = 0;
    while (offset < (size_t)num_candidates) {
        int this_batch = (int)min((size_t)batch_candidates, (size_t)num_candidates - offset);
        unsigned long long *d_cands = nullptr;
        int *d_counts = nullptr;
        CUDA_CHECK(cudaMalloc(&d_cands, (size_t)this_batch * M * sizeof(unsigned long long)));
        CUDA_CHECK(cudaMalloc(&d_counts, (size_t)this_batch * sizeof(int)));
        CUDA_CHECK(cudaMemset(d_counts, 0, (size_t)this_batch * sizeof(int)));
        CUDA_CHECK(cudaMemcpy(d_cands, cand_masks.data() + offset * M, (size_t)this_batch * M * sizeof(unsigned long long), cudaMemcpyHostToDevice));

        long long total_pairs = (long long)this_batch * (long long)num_transactions;
        int threads = 256;
        long long blocks = (total_pairs + threads - 1) / threads;
        if (blocks > 2147483647LL) {
            fprintf(stderr, "Too many blocks required; reduce batch size.\n");
            exit(EXIT_FAILURE);
        }
        count_kernel<<<(int)blocks, threads>>>(d_trans, d_cands, M, num_transactions, this_batch, d_counts);
        CUDA_CHECK(cudaGetLastError());
        CUDA_CHECK(cudaDeviceSynchronize());
        vector<int> batch_counts(this_batch);
        CUDA_CHECK(cudaMemcpy(batch_counts.data(), d_counts, (size_t)this_batch * sizeof(int), cudaMemcpyDeviceToHost));
        for (int i = 0; i < this_batch; ++i) counts[offset + i] = batch_counts[i];
        CUDA_CHECK(cudaFree(d_cands));
        CUDA_CHECK(cudaFree(d_counts));
        offset += this_batch;
    }
    CUDA_CHECK(cudaFree(d_trans));

    FrequentItemsets frequent_itemsets;
    for (int i = 0; i < num_candidates; ++i) {
        double support = (double)counts[i] / (double)num_transactions;
        if (support >= min_support) {
            frequent_itemsets[candidates[i]] = support;
        }
    }
    return frequent_itemsets;
}

FrequentItemsets apriori_gpu(const Transactions &transactions, double min_support) {
    FrequentItemsets all_frequent_itemsets;
    int k = 1;
    // prepare item->id and trans_masks
    set<Item> all_items_set;
    for (const auto &t : transactions) all_items_set.insert(t.begin(), t.end());
    vector<Item> all_items_vec(all_items_set.begin(), all_items_set.end());
    unordered_map<Item,int> item2id;
    for (int i = 0; i < (int)all_items_vec.size(); ++i) item2id[all_items_vec[i]] = i;
    int num_items = (int)all_items_vec.size();
    int M = (num_items + 63) / 64;
    int num_transactions = (int)transactions.size();
    vector<unsigned long long> trans_masks((size_t)num_transactions * M, 0ULL);
    for (int ti = 0; ti < num_transactions; ++ti) {
        for (const auto &it : transactions[ti]) {
            int id = item2id[it];
            int word = id / 64;
            int bit = id % 64;
            trans_masks[(size_t)ti * M + word] |= (1ULL << bit);
        }
    }

    vector<Itemset> current_candidates = create_candidate_itemsets(transactions, k);
    while (!current_candidates.empty()) {
        FrequentItemsets frequent_itemsets = filter_itemsets_gpu(transactions, current_candidates, min_support, item2id, M, trans_masks);
        if (frequent_itemsets.empty()) break;
        for (const auto &p : frequent_itemsets) all_frequent_itemsets[p.first] = p.second;
        ++k;
        set<Item> items_in_frequent;
        for (const auto &p : frequent_itemsets) items_in_frequent.insert(p.first.begin(), p.first.end());
        vector<Item> items_vec(items_in_frequent.begin(), items_in_frequent.end());
        vector<bool> bitmask(k, true);
        bitmask.resize(items_vec.size(), false);
        vector<Itemset> next_candidates;
        do {
            Itemset candidate;
            for (int i = 0; i < (int)items_vec.size(); ++i) if (bitmask[i]) candidate.insert(items_vec[i]);
            if ((int)candidate.size() == k) next_candidates.push_back(candidate);
        } while (prev_permutation(bitmask.begin(), bitmask.end()));
        current_candidates = next_candidates;
    }
    return all_frequent_itemsets;
}

vector<Itemset> generate_subsets(const Itemset &itemset) {
    vector<Item> items(itemset.begin(), itemset.end());
    vector<Itemset> subsets;
    int n = (int)items.size();
    for (int mask = 1; mask < (1 << n) - 1; ++mask) {
        Itemset subset;
        for (int i = 0; i < n; ++i) if (mask & (1 << i)) subset.insert(items[i]);
        subsets.push_back(subset);
    }
    return subsets;
}

Rules generate_rules(const FrequentItemsets &frequent_itemsets, double min_confidence) {
    Rules rules;
    for (const auto &pair : frequent_itemsets) {
        const Itemset &itemset = pair.first;
        if (itemset.size() < 2) continue;
        vector<Itemset> subsets = generate_subsets(itemset);
        double itemset_support = pair.second;
        for (const auto &antecedent : subsets) {
            Itemset consequent;
            set_difference(itemset.begin(), itemset.end(), antecedent.begin(), antecedent.end(), inserter(consequent, consequent.begin()));
            if (consequent.empty()) continue;
            auto it = frequent_itemsets.find(antecedent);
            if (it != frequent_itemsets.end()) {
                double antecedent_support = it->second;
                double confidence = itemset_support / antecedent_support;
                if (confidence >= min_confidence) rules.emplace_back(antecedent, consequent, itemset_support, confidence);
            }
        }
    }
    return rules;
}

string itemset_to_string(const Itemset &itemset) {
    string s = "{";
    bool first = true;
    for (const auto &item : itemset) {
        if (!first) s += ", ";
        s += item;
        first = false;
    }
    s += "}";
    return s;
}

int main(int argc, char** argv) {
    string csv_file;
    double min_support = 0.5, min_confidence = 0.7;
    if (argc >= 4) {
        csv_file = argv[1];
        min_support = atof(argv[2]);
        min_confidence = atof(argv[3]);
    } else {
        cout << "Enter CSV file name (e.g. dataset1.csv): ";
        getline(cin, csv_file);
        cout << "Enter minimum support (0-1): ";
        cin >> min_support;
        cout << "Enter minimum confidence (0-1): ";
        cin >> min_confidence;
    }

    Transactions transactions = read_transactions(csv_file);
    cout << "Loaded " << transactions.size() << " transactions." << endl;
    if (!transactions.empty()) {
        cout << "Sample transaction: ";
        for (const auto &item : transactions[0]) cout << item << " ";
        cout << endl;
    }
    FrequentItemsets frequent_itemsets = apriori_gpu(transactions, min_support);
    if (!frequent_itemsets.empty()) {
        cout << "\nFrequent Itemsets:" << endl;
        for (const auto &p : frequent_itemsets) {
            cout << itemset_to_string(p.first) << " -> support: " << p.second << endl;
        }
    } else {
        cout << "\nNo frequent itemsets found with the given minimum support." << endl;
    }
    Rules rules = generate_rules(frequent_itemsets, min_confidence);
    if (!rules.empty()) {
        cout << "\nAssociation Rules:" << endl;
        for (const auto &rule : rules) {
            cout << itemset_to_string(get<0>(rule)) << " => " << itemset_to_string(get<1>(rule))
                 << " (support: " << get<2>(rule) << ", confidence: " << get<3>(rule) << ")" << endl;
        }
    } else {
        cout << "\nNo association rules found with the given minimum confidence." << endl;
    }
    return 0;
}
EOF

echo "Wrote apriori_cuda.cu"


Wrote apriori_cuda.cu


##The Blow Command will Compile it with nvcc


In [ ]:
!nvcc -arch=sm_75 apriori_cuda.cu -o apriori_cuda


##Run the Compiled Code , the format of command is
```
codename <datasetfile_name> <min_support> <min_confidence>
```

In [ ]:
!./apriori_cuda realistic_dataset.csv 0.01 0.5
#The Format is like filename min_support and then min_confidence


Loaded 100000 transactions.
Sample transaction: laptop mouse keyboard 

Frequent Itemsets:
{baby_powder} -> support: 0.03517
{baby_powder, laptop} -> support: 0.03517
{baby_powder, laptop, wipes} -> support: 0.03517
{baby_powder, wipes} -> support: 0.03517
{bananas} -> support: 0.03452
{bananas, cereal} -> support: 0.03452
{bananas, cereal, milk} -> support: 0.03452
{bananas, milk} -> support: 0.03452
{beer} -> support: 0.05049
{beer, bread} -> support: 0.03558
{beer, bread, laptop} -> support: 0.03558
{beer, bread, laptop, milk} -> support: 0.03558
{beer, bread, milk} -> support: 0.03558
{beer, laptop} -> support: 0.05049
{beer, laptop, milk} -> support: 0.05049
{beer, milk} -> support: 0.05049
{bread} -> support: 0.38714
{bread, butter} -> support: 0.15111
{bread, butter, cheese} -> support: 0.03549
{bread, butter, jam} -> support: 0.03534
{bread, butter, milk} -> support: 0.0355
{bread, cheese} -> support: 0.07008
{bread, cheese, eggs} -> support: 0.03459
{bread, cheese, eggs, milk}

##Performance Improvements Noticed
-- Last Code Cell took 2 seconds to run and process the 1 Lakhs Record Dataset